In [1]:
#global path
global_drive_path="/content/drive/MyDrive/ForeCache/interactions/"
          

In [2]:
import os
import pandas as pd
import json
from pandas.io.json import json_normalize
from tqdm import tqdm

from statistics import mean
#for reward
import time
import datetime

pd.options.mode.chained_assignment = None  # default='warn'


In [3]:
def clean_files(filePath,fileName):

  lastline = None
 # filePath="/content/drive/Shareddrives/Colearning-Exploration/Dataset/forecache_interactions_2016/forecache-interaction-analysis/forecache-interaction-analysis/interactions/taskname_ndsi-2d-subtask_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.txt"
  with open(filePath,"r") as f:
      lineList = f.readlines()
      lastline=lineList[-1]
  with open(filePath,"r") as f, open(global_drive_path+"cleanInteractions/cleanInteractions-ndsi-2d-task/"+(str(fileName)+".json"),"w") as g:
  #with open(filePath,"r") as f, open(global_drive_path+"cleanInteractions/cleanInteractions-ndsi-3d-task/"+(str(fileName)+".json"),"w") as g:
  #os.makedirs(os.path.dirname(global_drive_path+"cleanInteractions/cleanInteractions-rgb-hurricane-3d-task/"+(str(fileName)+".json")))
  #with open(filePath,"r") as f, open(global_drive_path+"cleanInteractions/cleanInteractions-rgb-hurricane-3d-task/"+(str(fileName)+".json"),"w+") as g:
   
      for i,line in enumerate(f,0):
          if i == 0:
              line = "["+str(line)+","
              g.write(line)
          elif line == lastline:            
              g.write(line)
              g.write("]")
          else:
              line = str(line)+","
              g.write(line)
          

In [4]:
directory =(str(global_drive_path) +"/ndsi-2d-task")
#directory =(str(global_drive_path) +"/ndsi-3d-task")
#directory =(str(global_drive_path) +"/rgb-hurricane-3d-task")
failed_files=[]
for filename in os.listdir(directory):
  try:
    if filename.endswith(".txt"):
        
        filePath=(os.path.join(directory, filename))
        print(filename)
        name=os.path.splitext(filename)[0]
        clean_files(filePath,name)
    else:
        continue
  except OSError as err:
    print("OS error: {0}".format(err))
    failed_files.append(filename)
  except UnicodeDecodeError:
    print("Unicode Error")
    failed_files.append(filename)
  except BaseException as err:
    failed_files.append(filename)
    
    raise


taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714.txt
taskname_ndsi-2d-task_userid_ac5b65d7-f260-4534-9fae-d998b726ed32.txt
taskname_ndsi-2d-task_userid_82316e37-1117-4663-84b4-ddb6455c83b2.txt
taskname_ndsi-2d-task_userid_da52acdd-3cea-483c-86e5-2c2740d371aa.txt
taskname_ndsi-2d-task_userid_d6555293-35c7-4c27-b522-38d67d2d1b1a.txt
taskname_ndsi-2d-task_userid_ff56863b-0710-4a58-ad22-4bf2889c9bc0.txt
taskname_ndsi-2d-task_userid_bc9f9425-e867-4876-a5d9-791229916767.txt
taskname_ndsi-2d-task_userid_bda49380-37ad-41c5-a109-7fa198a7691a.txt
taskname_ndsi-2d-task_userid_7d014c75-453b-4188-9e4e-fb086984896a.txt
taskname_ndsi-2d-task_userid_8bf5b71b-dfd4-403f-8791-06427b26ccaf.txt
taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.txt
taskname_ndsi-2d-task_userid_6d49fab8-273b-4a91-948b-ecd14556b049.txt
taskname_ndsi-2d-task_userid_733a1ac5-0b01-485e-9b29-ac33932aa240.txt
taskname_ndsi-2d-task_userid_e4221e48-0930-40db-af3c-f4de1ed87582.txt
taskname_ndsi-2d-tas

In [5]:
#directory =(str(global_drive_path) +"/cleanInteractions//cleanInteractions-ndsi-3d-task")
directory =(str(global_drive_path) +"/cleanInteractions//cleanInteractions-ndsi-2d-task")
#directory =(str(global_drive_path) +"/cleanInteractions//cleanInteractions-rgb-hurricane-3d-task")
fileNames=[]
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        filePath=(os.path.join(directory, filename))
        fileNames.append(filename)
    else:
        continue

In [28]:
#  # SANITY CHECK !!! small view test
#  df = pd.read_json(os.path.join(directory,fileNames[0]))
#  df[:10]
#  df.toFetch[:10]
#  for i in range(len(df)):
#   print(df.toFetch[i])


{'dimindices': [0, 0], 'zoom': [0], 'name': 'zoom_0_pos_0_0'}
{'dimindices': [0, 0], 'zoom': [1], 'name': 'zoom_1_pos_0_0'}


IndexError: ignored

In [64]:
# # (pd.json_normalize(df.toFetch)).iloc[[0]][0][0]['name']
# df = pd.read_json(os.path.join(directory,fileNames[0]))
# ndsi=get_Ndsi(df)

'zoom_0_pos_0_0'

In [6]:
import numpy as np
import matplotlib.pyplot as plt
def angle_between(p1, p2):
    ang1 = np.arctan2(*p1[::-1])
    ang2 = np.arctan2(*p2[::-1])
    return np.rad2deg((ang1 - ang2) % (2 * np.pi))

In [13]:
def get_Ndsi(dataframe):
  zoom_files_global_path="/content/drive/MyDrive/ForeCache/ndsi_2d_jsons/ndsi_2d_jsons/" 
  toFetchData=pd.json_normalize(dataframe.toFetch)
  all_ndsi=[]
  for i in range(len(toFetchData)):
    current_row=toFetchData.iloc[i]
    row_ndsi=[]
    for j in range(len(current_row)):
      current_column=current_row[j]
      if current_column != None:
        zoom_file_name=current_column['name']
        path=zoom_files_global_path + str(zoom_file_name) +".json"
        try:
          with open(path) as data:    
            d = json.load(data)  
            df_zoom = pd.json_normalize(d)
            if df_zoom.domains[0][2][0] != None:
               column_mean_ndsi=mean(df_zoom.domains[0][2])
            else:
               column_mean_ndsi=0
        except FileNotFoundError:
          column_mean_ndsi=0
          print("Zoom file not found")
      row_ndsi.append(column_mean_ndsi)
    all_ndsi.append(mean(row_ndsi))
  return all_ndsi

In [8]:
def get_Angle(df):
  interactionData= pd.json_normalize(df.interactionData)
  x1=[]
  x2=[]
  y1=[]
  y2=[]
  angle=[]
  for i in range(len(interactionData['interactionType'])):
    #handles if the interactiontype doesn't have a coordinate : if interactionType is zoom
    if ((isinstance(interactionData['state.dragStart'][i], list) or isinstance(interactionData['state.dragEnd'][i], list) )==False):
      
      interactionData['state.dragStart'][i]=[0,0]
      interactionData['state.dragEnd'][i]=[0,0]
    #get between drag start and drag end
    angle.append(angle_between(interactionData['state.dragStart'][i],interactionData['state.dragEnd'][i]))
    #print(angle)

    #this are to plot later on
    x1.append(interactionData['state.dragStart'][i][0])
    y1.append(interactionData['state.dragStart'][i][1])

    x2.append(interactionData['state.dragEnd'][i][0])
    y2.append(interactionData['state.dragEnd'][i][1])
  return angle



In [9]:
#to generate reward based on radiclick and snapshot data 
def get_Reward(dataframe,filenamed):
  #global path
  reward=[]
  subtask=[]
  raw_timestamp= dataframe.timestampMillis
  snapshot_global_drive_path="/content/drive/MyDrive/ForeCache/forecache_user_study_data_dump/forecache_user_study_data_dump"
  #get a pandas dataframe object from json file
  path= snapshot_global_drive_path + "/snapshots.json"
  with open(path) as project_file:    
      data = json.load(project_file) 
  df_snapshots = pd.json_normalize(data)

  #get userid from fileName
  #filenamed[filenamed.find("userid_")+7: filenamed.find(".csv")] need to remove  hard code later
  dataframe_dimension=df_snapshots.loc[(df_snapshots['taskname']=="ndsi-2d-task") & (df_snapshots['userid']==filenamed[filenamed.find("userid_")+7: filenamed.find(".json")])] #change here for different Task name 2d, 3d
  print("There are ", len(dataframe_dimension) , "Snapshot times")
  for i in tqdm(range(len(raw_timestamp))):
      stringmyDate=str(raw_timestamp[i])
      myDate=stringmyDate[: stringmyDate.find(".")]
      dt = datetime.datetime.strptime(myDate, "%Y-%m-%d %H:%M:%S")
      a=time.mktime(dt.timetuple()) + (dt.microsecond / 1000000.0)
      temp=0
      task=1
      for j in range(len(dataframe_dimension["timestampMillis"])):


        if abs(a - (dataframe_dimension.timestampMillis.iloc[j]/1000)) < 10:
          temp=temp+10
        else:
         temp=temp+0
        if (a - (dataframe_dimension.timestampMillis.iloc[j]/1000)) < 0:
          task=task
        else:
          task=task+1
      subtask.append(task)
      reward.append(temp)
  return reward, subtask
    

In [10]:
def get_Zoom(df_zoom):
  df_zoom['state.newZoom'] = df_zoom['state.newZoom'].replace(np.nan, 0)
  lastZoom=[0,0]
  zoom_level=[]
  for i in tqdm(range(len(df_zoom))):
    if df_zoom['state.newZoom'][i]==0:
      if (lastZoom!=[0,0]):
          df_zoom['state.newZoom'][i]=lastZoom
      else:
        df_zoom['state.newZoom'][i]= [0,0]
        lastZoom= [0,0]
    else:
      lastZoom = df_zoom['state.newZoom'][i]
    zoom_level.append(df_zoom['state.newZoom'][i][0])
  return zoom_level

In [11]:
def get_IdleTime(df):
  idle_time=[]
  previous_time=df['timestampMillis'][0]
  for i in tqdm(range(len(df))):
      wait_time=df['timestampMillis'][i]-previous_time
      previous_time=df['timestampMillis'][i]


      #fixing format
      idle_time.append(wait_time.total_seconds())
  return idle_time

In [12]:
import heapq
def get_Subset_Idletime(idle_time):
  subset_idle_time=[]
  largest_indexes=heapq.nlargest(2, range(len(idle_time)), key=idle_time.__getitem__)
  second_max_index=largest_indexes[1]
  for j in tqdm(range(len(idle_time))):
    if (j<= second_max_index):
      subset_idle_time.append(1)
    else:
      subset_idle_time.append(2)
  return subset_idle_time

In [14]:
def get_State(df_state):
  states_array=[]
  for i in range(len(df_state)):
    if pd.isna(df_state.phase[i]):
      states_array.append('Answering')
    else:
      states_array.append(df_state.phase[i])
  return states_array

In [ ]:
# from collections import defaultdict
# probabilities=[]
# states_actions=[]
# _map = defaultdict(int)
# states=['Answering','SenseMaking','Foraging','Navigation']
# actions=['start','pan','zoom','radioClick']
# for state in states:
#   for action in actions:
#     states_actions.append(state+action)
#     _map[state+action]=1


In [15]:
from collections import defaultdict
def get_probabilities(df):
    probabilities=[]
    map=dict()

    states=['Answering','Sensemaking','Foraging','Navigation']
    actions=['start','pan','zoom','radioClick']
    for state in states:
      for action in actions:
        map[state+action]=1


    for i in range(len(df)):
      #print(str(df['State'][i]+df['Action']))
      if df['State'][i]=="Navigation":
        map[str(df['State'][i]+df['Action'][i])] += 1
        probs = (map[str(df['State'][i]+df['Action'][i])])/(map['Navigationpan'] + map['NavigationradioClick']+ map['Navigationstart'] + map['Navigationzoom'])

      elif df['State'][i]=="Sensemaking":
        map[str(df['State'][i]+df['Action'][i])] += 1
        probs = (map[str(df['State'][i]+df['Action'][i])])/(map['Sensemakingpan'] + map['SensemakingradioClick']+ map['Sensemakingstart'] + map['Sensemakingzoom'])
      elif df['State'][i]=="Foraging":
        map[str(df['State'][i]+df['Action'][i])] += 1
        probs = (map[str(df['State'][i]+df['Action'][i])])/(map['Foragingpan'] + map['ForagingradioClick']+ map['Foragingstart'] + map['Foragingzoom'])
      else:
        map[str(df['State'][i]+df['Action'][i])] += 1
        probs = (map[str(df['State'][i]+df['Action'][i])])/(map['Answeringpan'] + map['AnsweringradioClick']+ map['Answeringstart'] + map['Answeringzoom'])
      probabilities.append(probs)
    return probabilities

MAIN FUNCTION THAT RETURNS THE COMPLETED FILE

In [16]:
error_files=[]

for filename in tqdm(fileNames):
  try:
    #saveDir=(str(global_drive_path) +"/stateActionModel/ndsi-2d")
    saveDir=(str(global_drive_path) +"/stateActionModel/ndsi-2d")
    #saveDir=(str(global_drive_path) +"/stateActionModel/rgb-hurricane-3d")


    #get a pandas dataframe object from json file
    df = pd.read_json(os.path.join(directory,filename))
    print("Read file:",filename)
    interactionData=pd.json_normalize(df.interactionData)
    final_dataframe = pd.DataFrame(columns=['Action','State','TimeStamp','StateActionProbab','IdleTime','Reward','Subtask_Screenshot_Time','Subtask_Idle_Time','ZoomLevel','NDSI'])
    final_dataframe['NDSI']= get_Ndsi(df)	
    final_dataframe['Action']=interactionData.interactionType
    final_dataframe['State']=get_State(df)
    final_dataframe['TimeStamp']=df.timestampMillis
    final_dataframe['IdleTime']=get_IdleTime(df)
    final_dataframe['Subtask_Idle_Time']= get_Subset_Idletime(final_dataframe['IdleTime'])
    final_dataframe['Reward'],final_dataframe['Subtask_Screenshot_Time']= get_Reward(df,filename)
    final_dataframe['Angle']= get_Angle(df)
    final_dataframe['ZoomLevel']= get_Zoom(interactionData)	
    
    #get probabilities
    final_dataframe['StateActionProbab']=get_probabilities(final_dataframe)
    txt_version= filename.replace("json","csv")
    final_dataframe.to_csv(os.path.join(saveDir,txt_version), index=False)
  except AttributeError:
    print("AttributeError")
    error_files.append(filename)


  0%|          | 0/20 [00:00<?, ?it/s]

Read file: taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714.json
Zoom file not found
Zoom file not found
Zoom file not found
Zoom file not found
Zoom file not found
Zoom file not found
Zoom file not found
Zoom file not found
Zoom file not found
Zoom file not found
Zoom file not found
Zoom file not found
Zoom file not found
Zoom file not found



100%|██████████| 81/81 [00:00<00:00, 20985.77it/s]

100%|██████████| 81/81 [00:00<00:00, 316330.19it/s]


There are  3 Snapshot times



100%|██████████| 81/81 [00:00<00:00, 4435.23it/s]

  5%|▌         | 1/20 [00:01<00:29,  1.54s/it]

...


100%|██████████| 66/66 [00:00<00:00, 4720.42it/s]

 95%|█████████▌| 19/20 [00:55<00:01,  2.00s/it]

Read file: taskname_ndsi-2d-task_userid_93be0a02-3f7a-46c9-b526-aa956ed00856.json
Zoom file not found
Zoom file not found
Zoom file not found



100%|██████████| 130/130 [00:00<00:00, 11437.25it/s]

100%|██████████| 130/130 [00:00<00:00, 566209.26it/s]


There are  3 Snapshot times



100%|██████████| 130/130 [00:00<00:00, 3090.76it/s]

100%|██████████| 20/20 [00:57<00:00,  2.86s/it]


RANDOM TEST SNIPPETS AFTER THIS POINT

In [ ]:
len(matchingName) # so 19 users are the same 

NameError: ignored

In [ ]:
filenamed="taskname_ndsi-3d-task_userid_8b544d24-3274-4bb0-9719-fd2bccc87b02.csv"

In [ ]:
filenamed[filenamed.find("userid_")+7: filenamed.find(".csv")]

In [ ]:
df.head(5)

In [ ]:
df.phase

In [ ]:
interactionData= pd.io.json.json_normalize(df.interactionData)

In [ ]:
interactionData.interactionType

In [ ]:
final_dataframe = pd.DataFrame(columns=['Action','State','TimeStamp'])
final_dataframe['Action']=interactionData.interactionType
final_dataframe['State']=df.phase
final_dataframe['TimeStamp']=df.timestampMillis

In [ ]:
final_dataframe.head

In [ ]:
final_dataframe['Action']=interactionData.interactionType
final_dataframe['State']=df.phase
final_dataframe['TimeStamp']=df.timestampMillis

In [ ]:
final_dataframe

In [ ]:
for item in df.toFetch:
  print (item)

In [ ]:
toFetch

In [ ]:
import json
from pandas.io.json import json_normalize
normalized_json = json_normalize(df['taskname_ndsi-2d-subtask_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45'])
normalized_json.head(3 )


In [ ]:
normalized_json['toFetch']

In [ ]:
#get a pandas dataframe object from json file
#df = pd.read_json("/content/interactions_data/zoom_0_pos_0_0.json")
import json
import pandas as pd
from pandas.io.json import json_normalize
path= '/content/drive/Shareddrives/Colearning-Exploration/Dataset/forecache_interactions_2016/ndsi_2d_jsons/zoom_0_pos_0_0.json'
with open(path) as project_file:    
    data = json.load(project_file)  

df_zoom = pd.json_normalize(data)
df_zoom.head(5)

In [ ]:
df_zoom.domains